In [8]:
import time
import hopsworks
import os
from dotenv import load_dotenv
import joblib

# from datetime import datetime, timedelta
import pandas as pd

import hopsworks
import matplotlib.pyplot as plt

from utils import *
import utils


In [9]:
model_version = 4
fng_feature_group_version = 1
solana_feature_group_version = 1
bitcoin_feature_group_version = 1

In [10]:
today = int(time.time())
seconds_in_one_day = 24 * 60 * 60
today

1736011827

In [11]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login()
#secrets = secrets_api(project.name)
# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")

2025-01-04 19:30:27,826 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-04 19:30:27,828 INFO: Initializing external client
2025-01-04 19:30:27,829 INFO: Base URL: https://c.app.hopsworks.ai:443

Multiple projects found. 

	 (1) kth_lab1_2024
	 (2) ID2223_AirQuality_Lab1
2025-01-04 19:30:32,446 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164448


In [12]:
fs = project.get_feature_store() 
starting_timestamp = float(today - 40 * seconds_in_one_day)
starting_timestamp


1732555827.0

In [13]:
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=solana_feature_group_version
)

solana_batch_data = solana_feature_group.filter(solana_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
solana_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.68s) 


,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1732579200,233.96,239.68,221.82,230.68,445887.775346,201503.607457,244384.167889
1,1732665600,230.68,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184
2,1732752000,242.26,245.46,233.00,237.59,142466.637324,73142.398338,69324.238987
3,1732838400,237.59,247.04,236.58,243.47,301558.576846,121169.466633,180389.110213
4,1732924800,243.47,245.34,237.56,237.75,173213.101124,81975.271321,91237.829804


In [14]:
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=bitcoin_feature_group_version
)
bitcoin_batch_data = bitcoin_feature_group.filter(bitcoin_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
bitcoin_batch_data = bitcoin_batch_data[["timestamp","open"]]
bitcoin_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.64s) 


,timestamp,open
0,1732579200,93000.0
1,1732665600,91914.5
2,1732752000,95952.9
3,1732838400,95637.8
4,1732924800,97478.7


In [15]:
fng_feature_group = fs.get_feature_group(
    'f_n_g_index',
    version=fng_feature_group_version
)
fng_batch_data = fng_feature_group.filter(fng_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
fng_batch_data['fng_classification'] = fng_batch_data['fng_classification'].map(fng_labels_to_numbers_map)
fng_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 


,fng_value,fng_classification,timestamp
0,79,4,1.732579e+09
1,75,3,1.732666e+09
2,77,4,1.732752e+09
3,78,4,1.732838e+09
4,84,4,1.732925e+09


In [16]:
from tensorflow.keras.models import load_model

mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="solana_price_daily_model_10d",
    version=model_version,
)
saved_model_dir = retrieved_model.download()
print(saved_model_dir)
model = load_model(saved_model_dir+'/best_model.keras')
model


/var/folders/vc/103l9n915p1_fcsd6kw28dsm0000gn/T/403c1be8-41fa-4b06-8901-402ae9350f54/solana_price_daily_model_10d/4


<Sequential name=sequential_7, built=True>

In [17]:
all_data = pd.merge(solana_batch_data, bitcoin_batch_data, how='inner', on="timestamp",suffixes=('_solana', '_bitcoin'))  
all_data = pd.merge(all_data, fng_batch_data, how='inner', on="timestamp")

if "timestamp" in all_data.columns:
    all_data = all_data.drop(columns=['timestamp'])

target_column = 'open_solana'
all_data = utils.move_target_at_the_end(all_data, target_column)
all_data_values = all_data.values[-30:]

all_data.head()

,high,low,close,volume,volume_buy,volume_sell,open_bitcoin,fng_value,fng_classification,open_solana
0,239.68,221.82,230.68,445887.775346,201503.607457,244384.167889,93000.0,79,4,233.96
1,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184,91914.5,75,3,230.68
2,245.46,233.00,237.59,142466.637324,73142.398338,69324.238987,95952.9,77,4,242.26
3,247.04,236.58,243.47,301558.576846,121169.466633,180389.110213,95637.8,78,4,237.59
4,245.34,237.56,237.75,173213.101124,81975.271321,91237.829804,97478.7,84,4,243.47


In [18]:
model_workspace = "solana_price_10d_scalers/" 
scaler_labels = joblib.load(model_workspace+'scaler_labels.pkl')
scaler_features = joblib.load(model_workspace+'scaler_features.pkl')

all_data_values = scaler_features.transform(all_data_values)
X = all_data_values.reshape(1, 30, 10)


In [19]:
all_data.columns

Index(['high', 'low', 'close', 'volume', 'volume_buy', 'volume_sell',
       'open_bitcoin', 'fng_value', 'fng_classification', 'open_solana'],
      dtype='object')

In [20]:
predictions = model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


In [21]:
predictions_unscaled =  scaler_labels.inverse_transform(predictions)
predictions_unscaled


array([[212.51466, 222.72243, 188.07475, 216.17557, 206.06653, 206.62227,
        189.89925, 237.5527 , 214.14964, 212.27089]], dtype=float32)

In [22]:

timestamps = [today + (i+1) * seconds_in_one_day for i in range(len(predictions[0]))]

In [23]:
monitor_fg = fs.get_or_create_feature_group(
    name='solana_predictions',
    description='Solana_predictions',
    version=1,
    primary_key=['timestamp']
)

In [24]:
result = pd.DataFrame(columns=['timestamp', "open_solana"])
result["timestamp"]=timestamps
result["open_solana"]=predictions_unscaled[0]
#result= result.sort_values(by=['timestamp'])
result

,timestamp,open_solana
0,1736098227,212.514664
1,1736184627,222.722427
2,1736271027,188.074753
3,1736357427,216.175568
4,1736443827,206.066528
5,1736530227,206.622269
6,1736616627,189.899246
7,1736703027,237.552704
8,1736789427,214.149643
9,1736875827,212.270889


In [25]:
result['date'] = pd.to_datetime(result['timestamp'], unit='s')
result['date'] = result['date'].dt.normalize()
result

,timestamp,open_solana,date
0,1736098227,212.514664,2025-01-05
1,1736184627,222.722427,2025-01-06
2,1736271027,188.074753,2025-01-07
3,1736357427,216.175568,2025-01-08
4,1736443827,206.066528,2025-01-09
5,1736530227,206.622269,2025-01-10
6,1736616627,189.899246,2025-01-11
7,1736703027,237.552704,2025-01-12
8,1736789427,214.149643,2025-01-13
9,1736875827,212.270889,2025-01-14


In [26]:
#this is plane and we may have duplicates
#monitor_fg.insert(result)

In [27]:
#this insert prevents duplicates
monitor_fg.insert(
    result,
    # overwrite=False,  # Do not overwrite all data
    operation="upsert",  # Use upsert to handle duplicates based on the primary key
    #storage=None  # Write to both online and offline storage
)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1164448/fs/1155151/fg/1394537


Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: solana_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164448/jobs/named/solana_predictions_1_offline_fg_materialization/executions


(Job('solana_predictions_1_offline_fg_materialization', 'SPARK'), None)